In [1]:
import os

import openai
from openai import OpenAI

In [2]:
client = OpenAI(api_key=os.environ["OPENAI_KEY"])

In [3]:
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "This is my test request to ChatGPT"
        }
      ]
    }
  ],
)

In [4]:
response.choices[0].message.content

'Great! How can I assist you today?'